# Analýza sentimentu



Nastartujeme virtuální stroj Javy.

In [1]:
import weka.core.jvm as jvm
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\karel\\anaconda3\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\karel\\anaconda3\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\karel\\anaconda3\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'C:\\Users\\karel\\anaconda3\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'C:\\Users\\karel\\anaconda3\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


# Část Weka

Načtení originálních dat.

In [2]:
from weka.core.converters import Loader

loader = Loader("weka.core.converters.ArffLoader")
data = loader.load_file("./tweets.arff")

# nastaveni class atributu na první atribut - sentiment
data.class_index = 0

## Část bez úpravy předzpracování dat

In [3]:
from weka.filters import Filter

# aplikace filtru String To Word Vector - bez parametru
f = Filter(classname="weka.filters.unsupervised.attribute.StringToWordVector")
f.inputformat(data)
# vytvoreni filtrovaneho datasetu
expanded = f.filter(data)

Celkový počet vytvořených atributů z vektorizace:

In [4]:
print(len(expanded.attribute_names()))

1752


In [5]:
print(expanded)

@relation 'tweet_sentiment-weka.filters.unsupervised.attribute.StringToWordVector-R2-W1000-prune-rate-1.0-N0-stemmerweka.core.stemmers.NullStemmer-stopwords-handlerweka.core.stopwords.Null-M1-tokenizerweka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'

@attribute Sentiment {positive,negative}
@attribute 1 numeric
@attribute 10 numeric
@attribute 100 numeric
@attribute 1000 numeric
@attribute 123 numeric
@attribute 1700 numeric
@attribute 1st numeric
@attribute 2 numeric
@attribute 22 numeric
@attribute 226 numeric
@attribute 3 numeric
@attribute 4 numeric
@attribute 494 numeric
@attribute 5 numeric
@attribute 68 numeric
@attribute 710 numeric
@attribute 85832 numeric
@attribute A numeric
@attribute A320 numeric
@attribute AA numeric
@attribute AADFW numeric
@attribute AIRLINE numeric
@attribute AIRWAYS numeric
@attribute ALL numeric
@attribute AMAZING numeric
@attribute AMso numeric
@attribute AND numeric
@attribute ANYTHING numeric
@attribute ARE numeric
@a

In [6]:
f = Filter(classname="weka.filters.supervised.attribute.AttributeSelection")
f.inputformat(expanded)

final = f.filter(expanded)

Po provedení filtrace se vytvoří následující počet atributů:

In [7]:
print(len(final.attribute_names()))

83


In [8]:
print(final)

@relation 'tweet_sentiment-weka.filters.unsupervised.attribute.StringToWordVector-R2-W1000-prune-rate-1.0-N0-stemmerweka.core.stemmers.NullStemmer-stopwords-handlerweka.core.stopwords.Null-M1-tokenizerweka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"-weka.filters.supervised.attribute.AttributeSelection-Eweka.attributeSelection.CfsSubsetEval -P 1 -E 1-Sweka.attributeSelection.BestFirst -D 1 -N 5'

@attribute 10 numeric
@attribute 85832 numeric
@attribute AWESOME numeric
@attribute Cancelled numeric
@attribute Congratulations numeric
@attribute Deane numeric
@attribute Despite numeric
@attribute FOLLOW numeric
@attribute Great numeric
@attribute How numeric
@attribute Impressed numeric
@attribute JetBlue numeric
@attribute LAUREN numeric
@attribute Love numeric
@attribute No numeric
@attribute THANK numeric
@attribute Thank numeric
@attribute Thanks numeric
@attribute VirginAmerica numeric
@attribute amazing numeric
@attribute appreciate numeric
@attribute awe

In [63]:
from weka.classifiers import Classifier
from weka.classifiers import Evaluation
from weka.core.classes import from_commandline

classifiers = [
    'weka.classifiers.rules.ZeroR',
    'weka.classifiers.rules.OneR -B 6',
    'weka.classifiers.bayes.NaiveBayes',
    'weka.classifiers.trees.J48 -C 0.25 -M 2',
    'weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch"'
]

for classifier in classifiers:
    cls = from_commandline(classifier, classname="weka.classifiers.Classifier")
    cls.build_classifier(final)
    ev = Evaluation(final)
    ev.evaluate_train_test_split(cls, final, 66, rnd = None)
    
    print("Classifier " + classifier + " has correctly classfied " + str(ev.percent_correct) + " % instances")
    

Classifier weka.classifiers.rules.ZeroR has correctly classfied 88.91462607544672 % instances
Classifier weka.classifiers.rules.OneR -B 6 has correctly classfied 90.0397088021178 % instances
Classifier weka.classifiers.bayes.NaiveBayes has correctly classfied 93.61350099272005 % instances
Classifier weka.classifiers.trees.J48 -C 0.25 -M 2 has correctly classfied 92.81932495036399 % instances
Classifier weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 94.07677035076108 % instances


Využitím jiných algoritmů se samo o sobě přesnost zlepšila vcelku výrazně, nejlépe si zde vede K-nn algoritmus, následovaný NaiveBayes a neprořezaným rozhodovacím stromem. V další části provedu úpravy v předzpracování dat.

# Změna předzpracování dat

Postupně vyzkoušíme změnu předzpracování dat. V prvním kroku se vyzkouší odstranění stop slov, následně se vypíše počet atributů, vyfiltrují se atributy, natrénuje se klasifikátor a vypíše se výsledek.

Většina změn přístupu vede k velmi malým změnám, ale dohromady se jedná o zlepšení o cca. 5 procentních bodů oproti ZeroR. Ve srovnání s předchozím krokem ale není 

In [77]:
from weka.filters import Filter
from weka.filters import StringToWordVector

opt = [
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.NullStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # vychozi nastaveni + rainbow stopword handler
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.LovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # LovinsStemmer
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.LovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # vse nahore + TF
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.LovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # vse nahore + IDF,
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.LovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # vse nahore + TF-IDF
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "1", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.LovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # s normalizaci
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # tf-idf odlisny stemmer
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.NGramTokenizer -max 3 -min 1 -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ], # tokenizer vyuzivajici n-gramy
    [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.WordsFromFile -stopwords E:\Weka\Weka-3-8-4", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.NGramTokenizer -max 3 -min 1 -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
    ] # stop words from file
]

attr = [
    [
        "-E", "weka.attributeSelection.CfsSubsetEval -M -Z -P 1 -E 1", "-S", "weka.attributeSelection.BestFirst -D 2 -N 5"
    ]
   
]

m = 0

i = 1

for option in opt:
    print("---" + str(i) + "---")
    print(option)
    
    i = i + 1
    
    f = StringToWordVector(options = option)
    f.inputformat(data)

    vectorized = f.filter(data)

    print("SWV created attributes: " + str(len(vectorized.attribute_names())))

    f = Filter(classname="weka.filters.supervised.attribute.AttributeSelection", options=attr[0])
    f.inputformat(vectorized)
    toclassify = f.filter(vectorized)

    print("Filtered attributes: " + str(len(toclassify.attribute_names())))
    

    for classifier in classifiers:
        cls = from_commandline(classifier, classname="weka.classifiers.Classifier")
        cls.build_classifier(toclassify)
        ev = Evaluation(toclassify)
        ev.evaluate_train_test_split(cls, toclassify, 66, rnd = None)
        
        m = max(m, ev.percent_correct)

        print("Classifier " + classifier + " has correctly classfied " + str(ev.percent_correct) + " % instances")
        
    print("--------")

---1---
['-R', 'first-last', '-W', '1000', '-prune-rate', '-1.0', '-N', '0', '-L', '-stopwords-handler', 'weka.core.stopwords.Rainbow', '-stemmer', 'weka.core.stemmers.NullStemmer', '-M', '1', '-tokenizer', 'weka.core.tokenizers.WordTokenizer -delimiters " \\r\\n\\t.,;:\\\'\\"()?!"']
SWV created attributes: 3336
Filtered attributes: 92
Classifier weka.classifiers.rules.ZeroR has correctly classfied 88.91462607544672 % instances
Classifier weka.classifiers.rules.OneR -B 6 has correctly classfied 89.57643944407677 % instances
Classifier weka.classifiers.bayes.NaiveBayes has correctly classfied 90.23825281270682 % instances
Classifier weka.classifiers.trees.J48 -C 0.25 -M 2 has correctly classfied 90.27134348113832 % instances
Classifier weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 91.4295168762409 % instances
--------
---2---
['-R', 'first-last', '-W', '1000', '-prune-rate', '-1.0', '-N', '0', '-L', '-stopwords-handler', 'weka.

In [79]:
print(m)

94.3745863666446


Za nejlepší přístup se tedy nabízí kombinace stop words from file, ngram tokenizer, iterated Lovins stemmer, převod všeho na malá písmena. Z tohoto přístupu se zdá, že převod na tf-idf nemá žádný vliv na výsledek u těchto vybraných algoritmů. Může to být z toho důvodu, že méně častá slova budou odstraněna ve fázi redukce dimenzionality.

Nejlepší výsledek měl algoritmus NaiveBayes s úspěšností 94.3745863666446 %.

## NaiveBayes

V této části bude podstoupeno ladění metaparametrů NaiveBayes. NaiveBayes měl nejlepší výsledky u posledního pokusu z předchozí části, tak prvním krokem pro tento pokus bude využití předchozího předzpracování dat.

In [80]:
option = [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.WordsFromFile -stopwords E:\Weka\Weka-3-8-4", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.NGramTokenizer -max 3 -min 1 -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
]
    
f = StringToWordVector(options = option)
f.inputformat(data)

vectorized = f.filter(data)

print("SWV created attributes: " + str(len(vectorized.attribute_names())))

f = Filter(classname="weka.filters.supervised.attribute.AttributeSelection", options=attr[0])
f.inputformat(vectorized)
toclassify = f.filter(vectorized)

print("Filtered attributes: " + str(len(toclassify.attribute_names())))

classifiers_bayes = [
    "weka.classifiers.bayes.NaiveBayes",
    "weka.classifiers.bayes.NaiveBayes -K",
    "weka.classifiers.bayes.NaiveBayes -D"
]

for classifier in classifiers_bayes:
    cls = from_commandline(classifier, classname="weka.classifiers.Classifier")
    cls.build_classifier(toclassify)
    ev = Evaluation(toclassify)
    ev.evaluate_train_test_split(cls, toclassify, 66, rnd = None)

    m = max(m, ev.percent_correct)

    print("Classifier " + classifier + " has correctly classfied " + str(ev.percent_correct) + " % instances")

print("--------")


SWV created attributes: 1548
Filtered attributes: 59
Classifier weka.classifiers.bayes.NaiveBayes has correctly classfied 94.3745863666446 % instances
Classifier weka.classifiers.bayes.NaiveBayes -K has correctly classfied 95.16876240900066 % instances
Classifier weka.classifiers.bayes.NaiveBayes -D has correctly classfied 95.00330906684314 % instances
--------


Nejlepší výsledek NaiveBayes dosáhl s použitím parametru -D, tedy funkce využívající místo normálního rozdělení pravděpodobnosti KDE.

## IB-k

Clusterovací algoritmus dosáhl nejlepšího výsledku na předposledním výsledku předchozí části.

In [92]:
option = [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.Rainbow", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.NGramTokenizer -max 3 -min 1 -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
]
    
f = StringToWordVector(options = option)
f.inputformat(data)

vectorized = f.filter(data)

print("SWV created attributes: " + str(len(vectorized.attribute_names())))

f = Filter(classname="weka.filters.supervised.attribute.AttributeSelection", options=attr[0])
f.inputformat(vectorized)
toclassify = f.filter(vectorized)

print("Filtered attributes: " + str(len(toclassify.attribute_names())))

classifiers_ibk = [
    'weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 2 -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 3 -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 4 -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 4 -W 100 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 2 -I -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 2 -F -W 0 -A "weka.core.neighboursearch.LinearNNSearch"',
    'weka.classifiers.lazy.IBk -K 2 -W 0 -A "weka.core.neighboursearch.KDTree"',
    'weka.classifiers.lazy.IBk -K 2 -W 100 -A "weka.core.neighboursearch.BallTree"',
    'weka.classifiers.lazy.IBk -K 2 -W 100 -A "weka.core.neighboursearch.CoverTree"',
]

for classifier in classifiers_ibk:
    cls = from_commandline(classifier, classname="weka.classifiers.Classifier")
    cls.build_classifier(toclassify)
    ev = Evaluation(toclassify)
    ev.evaluate_train_test_split(cls, toclassify, 66, rnd = None)

    m = max(m, ev.percent_correct)

    print("Classifier " + classifier + " has correctly classfied " + str(ev.percent_correct) + " % instances")

print("--------")


SWV created attributes: 1734
Filtered attributes: 65
Classifier weka.classifiers.lazy.IBk -K 1 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 93.77895433487757 % instances
Classifier weka.classifiers.lazy.IBk -K 2 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 93.74586366644606 % instances
Classifier weka.classifiers.lazy.IBk -K 3 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 93.51422898742554 % instances
Classifier weka.classifiers.lazy.IBk -K 4 -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 93.51422898742554 % instances
Classifier weka.classifiers.lazy.IBk -K 4 -W 100 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 88.91462607544672 % instances
Classifier weka.classifiers.lazy.IBk -K 2 -I -W 0 -A "weka.core.neighboursearch.LinearNNSearch" has correctly classfied 93.71277299801456 % instances
Classifier weka.classifiers.lazy.IBk -K 2 -F -W 0 -A "weka.c

Veškeré provedené změny na tomto algoritmu vedly pouze ke zhoršení výsledků. Změna výpočtu vzdálenosti mezi instancemi bohužel nebyla vyzkoušena kvůli chybové hlášce ze strany knihovny při překopírování celého příkazu z prostředí Weka.

## J48

U tohoto algoritmu byl opět nejúspěšnější poslední výše uvedený případ.

In [96]:
option = [
        "-R", "first-last", "-W", "1000", "-prune-rate", "-1.0", "-N", "0", "-I", "-T", "-L",
        "-stopwords-handler", "weka.core.stopwords.WordsFromFile -stopwords E:\Weka\Weka-3-8-4", "-stemmer", "weka.core.stemmers.IteratedLovinsStemmer",
        "-M", "1", '-tokenizer', 'weka.core.tokenizers.NGramTokenizer -max 3 -min 1 -delimiters \" \\r\\n\\t.,;:\\\'\\\"()?!\"'
]
    
f = StringToWordVector(options = option)
f.inputformat(data)

vectorized = f.filter(data)

print("SWV created attributes: " + str(len(vectorized.attribute_names())))

f = Filter(classname="weka.filters.supervised.attribute.AttributeSelection", options=attr[0])
f.inputformat(vectorized)
toclassify = f.filter(vectorized)

print("Filtered attributes: " + str(len(toclassify.attribute_names())))

classifiers_tree = [
    "weka.classifiers.trees.J48 -C 0.25 -M 2",
    "weka.classifiers.trees.J48 -C 0.25 -M 5",
    "weka.classifiers.trees.J48 -R -N 3 -Q 1 -M 2",
    "weka.classifiers.trees.J48 -O -U -M 1"
]

for classifier in classifiers_tree:
    cls = from_commandline(classifier, classname="weka.classifiers.Classifier")
    cls.build_classifier(toclassify)
    ev = Evaluation(toclassify)
    ev.evaluate_train_test_split(cls, toclassify, 66, rnd = None)

    m = max(m, ev.percent_correct)

    print("Classifier " + classifier + " has correctly classfied " + str(ev.percent_correct) + " % instances")

print("--------")

SWV created attributes: 1548
Filtered attributes: 59
Classifier weka.classifiers.trees.J48 -C 0.25 -M 2 has correctly classfied 93.44804765056254 % instances
Classifier weka.classifiers.trees.J48 -C 0.25 -M 5 has correctly classfied 93.15023163467902 % instances
Classifier weka.classifiers.trees.J48 -R -N 3 -Q 1 -M 2 has correctly classfied 93.31568497683654 % instances
Classifier weka.classifiers.trees.J48 -O -U -M 1 has correctly classfied 93.05095962938451 % instances
--------


Výsledné stromy byly opět intervencí pouze zhoršeny. Lepší výsledek neměl ani neprořezaný strom.